In [1]:
import random

from deap import base
from deap import creator
from deap import tools

from PIL import Image, ImageChops
import numpy as np
import math, operator

C:\Users\Joe\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [ ]:
# set filename
# open image
# convert image to format ready to be compared to



In [ ]:
# create evaluate functions

# create mutate function - mutate floats, how to stay in range??

# create crossover function

In [79]:
from deap import base, creator, tools, algorithms
from random import randint, random, gauss
from PIL import Image, ImageDraw
from functools import partial
from math import sqrt
import numpy


PIC = Image.open('template.jpg')
SIZE = 100
NUMBER_OF_TRIANGLES = 5
POPULATION = 10
NGEN = 7
POLY = 3
VALUES_PER_GENE = 12

def computeSimilarity(allInds):
    numpyInds = numpy.array(allInds)
    groupedGenes = np.reshape(n.ravel(order='F'),(28,5))

    print(b)
    return 1


def gen_one_triangle():
    return [random() for i in range(VALUES_PER_GENE)]


def triangles_to_image(triangles):
    im = Image.new('RGB', (SIZE, SIZE), (255, 255, 255))
    for tri in triangles:
        mask = Image.new('RGBA', (SIZE, SIZE))
        draw = ImageDraw.Draw(mask)
        
        t = [int(x * SIZE) for x in tri[0:6]]
        triangle = ((t[0],t[1]),(t[2],t[3]),(t[4],t[5]))
        rgb = [int(x * 255) for x in tri[3:6]]
        fillAlpha = int(tri[5] * 30)

        fillColor = (rgb[0],rgb[1],rgb[2],fillAlpha)
        draw.polygon(triangle, fill=fillColor)
        im.paste(mask, mask=mask)
        del mask, draw
    return im


def evaluate(im1, t2):
    im2 = triangles_to_image(t2)

    h = ImageChops.difference(im1,im2).histogram()
    length = int(len(h)/3)

    r = h[length*0:length*1]
    g = h[length*1:length*2]
    b = h[length*2:length*3]

    h = [sum(x) for x in zip(r,g,b)]
    err = math.sqrt(sum(h*(i**2) for i, h in enumerate(h)) / float(SIZE * SIZE * 3)) / 256
    return 1 - err,



def mutate(triangles):
    for i in range(2):
        triangleToMutate = triangles[randint(0,len(triangles)-1)]
        valueIndexToMutate = randint(0,len(triangleToMutate)-1)
        triangleToMutate[valueIndexToMutate] = random()
        
    return triangles,

creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.Fitness)

toolbox = base.Toolbox()
toolbox.register("attr", gen_one_triangle)
toolbox.register("individual", tools.initRepeat,
                 creator.Individual, toolbox.attr, NUMBER_OF_TRIANGLES)
toolbox.register("population", tools.initRepeat,
                 list, toolbox.individual)

toolbox.register("evaluate", partial(evaluate, PIC))
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)


def main():
    pop = toolbox.population(n=POPULATION)
    hof = tools.HallOfFame(1)

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_sim = tools.Statistics(lambda ind: ind)
    stats_fit.register("std", numpy.std)
    stats_fit.register("max", numpy.max)
    stats_fit.register("avg", numpy.mean)
    stats_fit.register("min", numpy.min)
    stats_sim.register("similarity",computeSimilarity)

    mstats = tools.MultiStatistics(fitness=stats_fit, similarity=stats_sim)

    
    try:
        pop, log = algorithms.eaSimple(
            pop, toolbox, cxpb=0.5, mutpb=0.1, ngen=NGEN, stats=mstats,
            halloffame=hof, verbose=True)
    finally:
        open('stats.txt', 'w').write(str(log))
        open('result.txt', 'w').write(repr(hof[0]))
        triangles_to_image(hof[0]).save('result.png')
        
if __name__ == '__main__':
    main()

[[[ 0.35062877  0.82297492  0.6519858   0.74501465  0.16695933  0.9186203
    0.72362606  0.26326859  0.84448635  0.36817565  0.79749555  0.90517536]
  [ 0.71134642  0.12711997  0.89085791  0.1269591   0.56337178  0.19219583
    0.65448215  0.65257524  0.88357177  0.75248592  0.8105864   0.5334378 ]
  [ 0.1259017   0.90822863  0.71835107  0.79634626  0.85404329  0.13214214
    0.72382157  0.42423175  0.28225527  0.90881022  0.5538786   0.51019263]
  [ 0.42684433  0.50703756  0.4767756   0.97630308  0.3153554   0.14677634
    0.59991476  0.88805679  0.84038415  0.40254528  0.54357527  0.9247936 ]
  [ 0.5916966   0.73818386  0.02097525  0.93793728  0.17329175  0.60048518
    0.87893796  0.13025777  0.26115864  0.87976359  0.10654706  0.80148049]]

 [[ 0.13930313  0.10794036  0.00385439  0.64602475  0.50266219  0.99079429
    0.81061762  0.06474455  0.2045177   0.8557275   0.94686416  0.87267187]
  [ 0.65000186  0.07573726  0.2213976   0.94711729  0.24927914  0.80782203
    0.82982979  0.

C:\Users\Joe\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Joe\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [65]:
import cProfile
cProfile.run("main()")

gen	nevals	std       	max     	avg     	min     
0  	75    	0.00913407	0.501332	0.470455	0.448969
1  	33    	0.00881925	0.501591	0.477101	0.458309
2  	41    	0.00888577	0.508514	0.484942	0.470293
3  	44    	0.00886675	0.507653	0.491934	0.476185
4  	33    	0.00636476	0.509895	0.499001	0.472964
5  	42    	0.00404629	0.51305 	0.503793	0.492986
6  	50    	0.00355603	0.51509 	0.506916	0.499301
7  	44    	0.00330504	0.517001	0.509833	0.501313
8  	45    	0.00290689	0.520649	0.513052	0.504608
9  	38    	0.00258472	0.523363	0.515206	0.507007
10 	38    	0.00290661	0.523754	0.517078	0.51055 
11 	36    	0.00253039	0.523754	0.519224	0.514816
12 	38    	0.00257395	0.529409	0.521477	0.512251
13 	52    	0.00141759	0.52711 	0.523093	0.517693
14 	42    	0.00165115	0.528553	0.524173	0.517957
15 	42    	0.00156004	0.530213	0.525474	0.521445
16 	38    	0.00129205	0.5304  	0.526495	0.523139
17 	28    	0.00166535	0.5304  	0.527711	0.521595
18 	33    	0.00122165	0.532242	0.528972	0.525634
19 	48    	0.0009139

In [95]:

arr = []
for indNum in range(5):
    indA = []
    for triNum in range(4):
        triA = []
        for geneNum in range(7):
            s = str(indNum) +"-" +str(triNum) +"-"+str(geneNum)
            triA.append(s)
        indA.append(triA)
    arr.append(indA)
    
n = numpy.array(arr)

n.ravel(order='F')
np.reshape(n.ravel(order='F'),(28,5))
#n.reshape((5,4,7),order='F')

array([['0-0-0', '1-0-0', '2-0-0', '3-0-0', '4-0-0'],
       ['0-1-0', '1-1-0', '2-1-0', '3-1-0', '4-1-0'],
       ['0-2-0', '1-2-0', '2-2-0', '3-2-0', '4-2-0'],
       ['0-3-0', '1-3-0', '2-3-0', '3-3-0', '4-3-0'],
       ['0-0-1', '1-0-1', '2-0-1', '3-0-1', '4-0-1'],
       ['0-1-1', '1-1-1', '2-1-1', '3-1-1', '4-1-1'],
       ['0-2-1', '1-2-1', '2-2-1', '3-2-1', '4-2-1'],
       ['0-3-1', '1-3-1', '2-3-1', '3-3-1', '4-3-1'],
       ['0-0-2', '1-0-2', '2-0-2', '3-0-2', '4-0-2'],
       ['0-1-2', '1-1-2', '2-1-2', '3-1-2', '4-1-2'],
       ['0-2-2', '1-2-2', '2-2-2', '3-2-2', '4-2-2'],
       ['0-3-2', '1-3-2', '2-3-2', '3-3-2', '4-3-2'],
       ['0-0-3', '1-0-3', '2-0-3', '3-0-3', '4-0-3'],
       ['0-1-3', '1-1-3', '2-1-3', '3-1-3', '4-1-3'],
       ['0-2-3', '1-2-3', '2-2-3', '3-2-3', '4-2-3'],
       ['0-3-3', '1-3-3', '2-3-3', '3-3-3', '4-3-3'],
       ['0-0-4', '1-0-4', '2-0-4', '3-0-4', '4-0-4'],
       ['0-1-4', '1-1-4', '2-1-4', '3-1-4', '4-1-4'],
       ['0-2-4', '1-2-4', '2